In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [24]:
class TwitterClient(object):
#Generic Twitter class for Sentiment analysis
    def __init__(self):
#Class Constructor
#Keys and tokens from the Twitter Application
        consumer_key = 'KD3hVawYuy54NLQqvEL2wLlBx'
        consumer_secret = 'e0TCPUqnWGIHbxpZPf23TWdYVcZPI8zz3trkjMQ4IWBPlcdSHi'
        access_token = '349087376-yVmTxa64fKjtH9vvc3eTcbgsxVv2r0v1NICmq6yF'
        access_token_secret = 'pRZ5x5iKPjqyC4QS0CG7zjb9akWNjwG5ghhb7gqO5iiPk'
        try:
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
            
    def clean_tweet(self, tweet):
        #Tweet cleaning to remove hatags and other special characters
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\?\?\s+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        #Function to classify sentiments of passed tweets using TextBlob's sentiment method
        analysis = TextBlob(self.clean_tweet(tweet))
        #set sentiments
        if analysis.sentiment.polarity > 0:
            return 'Positive'
        elif analysis.sentiment.polarity == 0:
            return ' Neutral'
        else:
            return 'Negative'

    def get_tweets(self, query, count = 1000):
        tweets = []
        try:
            fetched_tweets = self.api.search(q = query, count = count)
            for tweet in fetched_tweets:
                    # empty dictionary to store required params of a tweet
                    parsed_tweet = {}

                    # saving text of tweet
                    parsed_tweet['text'] = tweet.text
                    # saving sentiment of tweet
                    parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                    # appending parsed tweet to tweets list
                    if tweet.retweet_count > 0:
                        # if tweet has retweets, ensure that it is appended only once
                        if parsed_tweet not in tweets:
                            tweets.append(parsed_tweet)
                    else:
                        tweets.append(parsed_tweet)

                # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))        

In [25]:
def main():
    api = TwitterClient()
    tweets = api.get_tweets(query = '#Chelsea', count = 500) 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'Positive']
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'Negative']
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    print("\n\nPositive tweets:")
    for tweet in ptweets[:5]:
        print(tweet['text'])
    print("\n\nNegative tweets:")
    for tweet in ntweets[:5]:
        print(tweet['text'])
 
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 31.818181818181817 %
Negative tweets percentage: 6.818181818181818 %


Positive tweets:
PL Clubs, Be Scared! Be Very Scared! 👻 #Chelsea https://t.co/m7LV5ViP45
RT @ODonnellDale: I’d imagine Sarri’s smoking intake would increase significantly with the stress that #Chelsea dressing room would bring.…
@Chelsea_FL .@ChelseaFC announce that they are ready to meet Luis Enrique's requirements for tea in the morning and… https://t.co/TRouQ5YxwT
FIFA 18 TOTW 21 confirmed: FUT Team of the Week revealed – Salah, Mkhitaryan, Walc #Chelsea #CFC #FNH https://t.co/rtisq8JIGP
Luis Suarez to Chelsea: How Barcelona superstar could fit into Blues XI #Chelsea #CFC #FNH https://t.co/LYOUTUhwUl


Negative tweets:
RT @tribalfootball: Breaking down the Chelsea crisis: What’s happened and what’s next for Antonio Conte? #CFC #Chelsea #Conte @Asif9707  ht…
Time to get the ball rolling this #Mueller #Trump Collusion bullshit has gone on long enough. The #Treason of the… https://t.co/2o